In [6]:
import numbers
from sklearn.preprocessing import minmax_scale
from numpy.random import randint
import numpy as np
import pandas as pd

In [7]:
Df = pd.read_csv("Données/bank.csv",sep=";")

In [8]:
np.array(list(list(Df[i]) for i in Df))[0,:]

array(['30', '33', '35', ..., '57', '28', '44'], dtype='<U13')

In [10]:
Df.iloc[[0,1,4]]['age']

0    30
1    33
4    59
Name: age, dtype: int64

In [9]:
isinstance(6.8,numbers.Real)

True

In [11]:
np.array([[0,2,0],[2,1,1]]).min(axis=0)

array([0, 1, 0])

In [12]:
def dissimilarity(pointA,pointB):
    """
    Renvoie l'indice de dissimilarité 
    Plus il est élevé et plus la distance entre les deux points est grande
    """
    
    res = sum([1 for k in pointA.index if pointA[k] != pointB[k]])
    
    return res

In [57]:
def new_centroid(cluster,numerical_keys,categorical_keys):
    """
    Renvoie le centre d'un cluster catégorical
    """
    # Calcul des coordonnées numériques du nouveau centre du cluster
    centroid = [cluster[k].mean() for k in numerical_keys]
     
    # Calcul des coordonnées catégoriques du nouveau centre du cluster    
    for k in categorical_keys :
        values = dict((i,cluster[cluster[k]==i].shape[0]) for i in set(cluster[k])) 
        # On prend la valeur qui apparait le plus souvent comme centre
        centroid.append(max(values,key=values.get))
        
    serie = pd.DataFrame(data=[centroid],columns=numerical_keys+categorical_keys)
        
    return serie

In [58]:
new_centroid(Df.iloc[0:10],n,c)

,age,balance,day,duration,campaign,pdays,previous,job,marital,education,default,housing,loan,contact,month,poutcome,y
0,38.1,1073.6,12.8,191.2,1.6,131.7,1.2,management,married,tertiary,no,yes,no,cellular,may,unknown,no


In [68]:
def KPrototype(K,data,max_iter,gamma):
    """
    
    """
    # Tri des clés ayant des valeurs catégoriques et numériques
    number_keys = []
    categorical_keys = []
    
    for k in data :
        if isinstance(data.iloc[0][k],numbers.Real) :
            number_keys.append(k)
        else:
            categorical_keys.append(k)
    
    # Préparation des 2 ensembles de données
    categorical = data[categorical_keys]
    numerical = pd.DataFrame(minmax_scale(np.array(list(list(data[k]) for k in number_keys))).T,columns=number_keys)
    
    dataset = pd.concat([numerical,categorical],axis=1)
    centers = dataset.iloc[randint(0,data.shape[0],K)] 
    
    i = 0
    not_centers_same = True
    
    while (i!=max_iter and not_centers_same):
        label = []
        i+=1
        
        # Remise à zéro des clusters
        cluster = dict((i,pd.DataFrame(columns=(number_keys+categorical_keys))) for i in range(K))
        
        for d in range(data.shape[0]):
            value_clusters = []
            for i in range(K):
                distance = np.linalg.norm(np.array(numerical.iloc[d])-np.array(centers.iloc[i][number_keys]))+gamma*dissimilarity(centers.iloc[i][categorical_keys],categorical.iloc[d])
                value_clusters.append((distance,i))
            
            minimum_center = min(value_clusters)
            cluster[minimum_center[1]].loc[len(cluster[minimum_center[1]])] = dataset.iloc[d]
            label.append(minimum_center[1])
          
        
        # Recalcule les centres
        new_centers= [new_centroid(cluster[i],number_keys,categorical_keys) for i in range(K)]
        new_centers = pd.concat(new_centers,axis=0)
        
        if (np.array_equal(new_centers,centers)): 
            not_centers_same = False
        else:
            centers=new_centers
        
    return (new_centers,cluster,label) 

** Remarque: quand les données ne sont pas normalisé, les clusters ne sont pas équilibré, il y a tout dans un même cluster **
- Revoir: choix des points aléatoire au début

In [69]:
ncenters,cluster,label = KPrototype(2,Df,1000,1)